In [1]:
import pandas as pd
import numpy as np
import warnings
from gensim.models import Word2Vec
from mittens import GloVe
from sklearn.metrics.pairwise import cosine_similarity
from mittens import GloVe
from sklearn import metrics
warnings.filterwarnings("ignore")

In [2]:
#Load UK dataset
UK_News = pd.read_pickle(r'UK_News_SOM_Bubble_final_bestModel.df')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [455]:
#Subset only tweets that were humanly labeled
UK_News_V = UK_News.loc[UK_News['True Topic'] != '']
len(UK_News_V)

7167

During the labeling process were used the following labels 
Label1-covid, health
Labe2-election, usa, trump, biden,usa politics
Label3- crimes
Label4-royals
Label4_2-celeb
Label5-not used. Wanted to be used for 'Life&People' topic but because of time consuming of the labeling process at the end it was not used but in the future can be used for such 
topic
Label6- sport
Label7-brexit
Label8-space
Label8_2-tech, digital companies. It has been calles 8_2 instead of just 9 because during 
the labeling process initially there was some indecision if to put it together with Label8
Label9-politics
Label10-economy, financial, markets
Label11-not used, the same as fo Label5 
Label12- international affairs, war, conflicts
Label13- meteo, environment conditions 


# Gensim tunning - mean clusters 'in' similarity 
For each trained Gensim Models was computed (steps):
1. For each cluster the similarity of each cluster tweet vs all others cluster tweets
2. For each cluster the mean of all cluster tweets similarites (computed in the first step) 
3. The mean of all clusters mean similarities (computed into second step) - called 'Similarity A'

NB: the term 'cluster' intended as 'topic'

In [452]:
from gensim.models import Word2Vec
# train models
model_gens50_4= Word2Vec(UK_News['Tweets_C_tok12'], min_count=0,size=50,window=4)
model_gens50_12= Word2Vec(UK_News['Tweets_C_tok12'], min_count=0,size=50,window=12)
model_gens25_8= Word2Vec(UK_News['Tweets_C_tok12'], min_count=0,size=25,window=8)
model_gens100_8= Word2Vec(UK_News['Tweets_C_tok12'], min_count=0,size=100,window=8)
#fuction to apply the models embeddings to tweets words and compute the mean of all
#vector words by which eah tweet is composed of.
def text2vecGens1(text):
    return np.mean([model_gens50_4[x] for x in text if x in model_gens50_4.wv.vocab], axis=0).reshape(1,-1)
def text2vecGens2(text):
    return np.mean([model_gens50_12[x] for x in text if x in model_gens50_12.wv.vocab], axis=0).reshape(1,-1)
def text2vecGens3(text):
    return np.mean([model_gens25_8[x] for x in text if x in model_gens25_8.wv.vocab], axis=0).reshape(1,-1)
def text2vecGens4(text):
    return np.mean([model_gens100_8[x] for x in text if x in model_gens100_8.wv.vocab], axis=0).reshape(1,-1)

In [453]:
gens_models_list=[ text2vecGens1,text2vecGens2,text2vecGens3,text2vecGens4]
labels_list=['Label1','Label2','Label3','Label4','Label4_2','Label6','Label7','Label8','Label8_2','Label9',
            'Label10','Label12','Label13']
simil_means_GensimModels_all=[]
for j in gens_models_list:
    UK_News_V['meanWordsGensXX_xx'] = UK_News_V['Tweets_C_tok12'].apply(j)
    for i in labels_list:
        Tweets_Label=UK_News_V[UK_News_V['True Topic']==i]
        Tweets_Vec_Gensim_lab= np.concatenate(Tweets_Label['meanWordsGensXX_xx'].values)
        gensim_lab_CosMatrix=metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab, Y=None, dense_output=True)
        gensim_lab_CosMatrix2=gensim_lab_CosMatrix[np.triu_indices(len(Tweets_Label), k = 1)]
        mean_lab=np.mean(gensim_lab_CosMatrix2)
        simil_means_GensimModels_all.append(mean_lab) 

###  ' Similarity A'

In [454]:
simil_means_GensimModels_50_4=(simil_means_GensimModels_all[0:13])
simil_means_GensimModels_50_12=(simil_means_GensimModels_all[13:13+13])
simil_means_GensimModels_25_8=(simil_means_GensimModels_all[26:26+13])
simil_means_GensimModels_100_8=(simil_means_GensimModels_all[39:39+13])
simil_means2_GensimModels_all=[]
for i in [simil_means_GensimModels_50_4,simil_means_GensimModels_50_12,simil_means_GensimModels_25_8,simil_means_GensimModels_100_8]:
    simil_means2_GensimModels_all.append(round(mean(i),2))
for i,j in zip(simil_means2_GensimModels_all,['50,  4','50, 12','25,  8','100, 8']):
    print('Overall Similarity for Gensim model size and window equal to:',j,'  is: ',round(i*100), '%')

Overall Similarity for Gensim model size and window equal to: 50,  4   is:  79 %
Overall Similarity for Gensim model size and window equal to: 50, 12   is:  78 %
Overall Similarity for Gensim model size and window equal to: 25,  8   is:  79 %
Overall Similarity for Gensim model size and window equal to: 100, 8   is:  79 %


# Glove

Glove model is performed with mittens having the choice to train the model with using/or not and in which measure to use the pretrained Glove wrod embeddings. 
Mittens Glove requires the co-occurences words matrix, have been performed 2 cases:
- case 1 the co-occurence matrix computed by using the TfidfVectorizer 
- case 2 the co-occurence matrix computed by using the CountVectorizer

Mittens Glove was performed with several parameters values (dimension, mittens, co-occurences -> 'case 1' and 'case 2'. They have been performed one at time and have been taken note about their performance. The last Mittens Glove, thus to which some of outputs of the following code belong, is with 'case 2', dimension 25 and mittens param 0.1

#### Co-occurences matrix computation

In [467]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#case 1
tfidf2_UKNews_Glove= TfidfVectorizer(tokenizer=lambda x: x,preprocessor=lambda x: x,use_idf=False,smooth_idf=False) 
dtm2_UKNews_Glove= tfidf2_UKNews_Glove.fit_transform(UK_News['Tweets_C_tok12'])
print(len(tfidf2_UKNews_Glove.get_feature_names()))

4352


In [14]:
#case 1
X=dtm2_UKNews_Glove
X[X > 0] = 1
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format

In [469]:
#case 2
count_model = CountVectorizer(tokenizer=lambda x: x,preprocessor=lambda x: x,ngram_range=(1,1)) # default unigram model
X2 = count_model.fit_transform(UK_News['Tweets_C_tok12'])
X2[X2 > 0] = 1
X2c=(X2.T*X2)

In [765]:
#with Tf-idf Vectorizer
cooccurrence1=Xc.toarray()
vocab1=tfidf2_UKNews_Glove.vocabulary_
#with CountVectorizer
cooccurrence2=X2c.toarray()
vocab2=count_model.vocabulary_

#### Loading pre trained Glove - word vectors with dim 25 and 50 on Twitter data 

In [15]:
import csv
def glove2dict(glove_filename):
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

In [766]:
original_embeddings25=glove2dict('glove.twitter.27B.25d.txt')

In [118]:
original_embeddings50=glove2dict('glove.twitter.27B.50d.txt')

In [312]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [316]:
glove_emb_tweet_50 = load_embed('glove.twitter.27B.50d.txt')
import operator 
print("Glove : ")
oov_glove = check_coverage(vocab, glove_emb_tweet_50 )

Glove : 
Found embeddings for 97.89% of vocab
Found embeddings for  97.77% of all text


#### Mittens Glove model definition and training 

In [810]:
from mittens import Mittens
mittens_parm=0.1
mittens_model = Mittens(n=25, max_iter=2000,mittens=mittens_parm)
# Note: n must match the original embedding dimension
new_embeddings = mittens_model.fit(
    cooccurrence2,
    vocab=vocab2,
    initial_embedding_dict= original_embeddings25)

Iteration 2000: loss: 26383.060546875

##### Some code to obtain cosine similarities 

In [811]:
feature_vec_1=new_embeddings[vocab['restriction']]
#feature_vec_2=new_embeddings[vocab['economic']]
def get_cosine_similarity2(feature_vec_1):
    list_simil=[]
    for i in vocab:
        list_simil.append(cosine_similarity(feature_vec_1.reshape(1, -1), new_embeddings[vocab[i]].reshape(1, -1))[0][0])
    return list_simil   
list_worsVocab=[]
for i in vocab:
    list_worsVocab.append(i)
cosine_sim=dict(zip(list_worsVocab, get_cosine_similarity2(feature_vec_1)))
#get_cosine_similarity2(feature_vec_1)
sorted(cosine_sim.items(), key=lambda x: x[1], reverse=True)[1:40]
feature4_vec_1=new_embeddings[vocab['coronavirus']]
feature4_vec_2=new_embeddings[vocab['restriction']]
def get_cosine_similarity(feature4_vec_1, feature4_vec_2):    
    return cosine_similarity(feature4_vec_1.reshape(1, -1), feature4_vec_2.reshape(1, -1))[0][0]
get_cosine_similarity(feature4_vec_1, feature4_vec_2)

0.75814456

#### Covert each tweet in vector by doing the mean of its words vectors embedded with mittens glove trained model (as in the case of gensim)

In [812]:
def text2vecGlove(text):
    return np.mean([new_embeddings[vocab[x]] for x in text if x in vocab], axis=0).reshape(1,-1)
UK_News_V['meanWordsGlove_tok_12'] = UK_News_V['Tweets_C_tok12'].apply(text2vecGlove)

### End of Mittens Glove model settings and processing spets

In [814]:
#applying the several trained gensim models in order to be used when will be computed next
#the similirity lab X vs all
UK_News_V['meanWordsGens50_4'] = UK_News_V['Tweets_C_tok12'].apply(text2vecGens1)
UK_News_V['meanWordsGens50_12'] = UK_News_V['Tweets_C_tok12'].apply(text2vecGens2)
UK_News_V['meanWordsGens25_8'] = UK_News_V['Tweets_C_tok12'].apply(text2vecGens3)
UK_News_V['meanWordsGens100_8'] = UK_News_V['Tweets_C_tok12'].apply(text2vecGens4)

## Gensim  and Glove - Cosine similarities
Since the training of mittens Glove model takes some time -> a mittens Glove model configuration (with a certain value for  mittens parameter) had been run one at time e then the cosine similarities had been computed and taken note about the results. The output of the following cells refers in the case of Glove to the last Mittens Glove fitted with 'case 2' co-occurences matrix, dimension 25 and mittens param 0.1, and in the case of Gensim to the model used in the clustering analysis with window=8 and dim=50.

#### Retrieving texts and their vectors for each topic separately

In [834]:
gensim_embedXX='meanWordsGens_tok_12' #'meanWordsGens_tok_12' -> used for embedding in all clustering models 
#Select tweets ->lab 6
Tweets_Label6=UK_News_V[UK_News_V['True Topic']=='Label6']
Tweets_Vec_Glove_lab6= np.concatenate(Tweets_Label6['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab6= np.concatenate(Tweets_Label6[gensim_embedXX].values)
# #Select  tweets ->lab 8
Tweets_Label8=UK_News_V[UK_News_V['True Topic']=='Label8']
Tweets_Vec_Glove_lab8= np.concatenate(Tweets_Label8['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab8= np.concatenate(Tweets_Label8[gensim_embedXX].values)
# #Select  tweets ->lab 13
Tweets_Label13=UK_News_V[UK_News_V['True Topic']=='Label13']
Tweets_Vec_Glove_lab13= np.concatenate(Tweets_Label13['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab13= np.concatenate(Tweets_Label13[gensim_embedXX].values)
# #Select tweets ->lab 9
Tweets_Label9=UK_News_V[UK_News_V['True Topic']=='Label9']
Tweets_Vec_Glove_lab9= np.concatenate(Tweets_Label9['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab9= np.concatenate(Tweets_Label9[gensim_embedXX].values)
# #Select some tweets ->lab 4_2
Tweets_Label4_2=UK_News_V[UK_News_V['True Topic']=='Label4_2']
Tweets_Vec_Glove_lab4_2= np.concatenate(Tweets_Label4_2['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab4_2= np.concatenate(Tweets_Label4_2[gensim_embedXX].values)
# #Select some tweets ->lab 4
Tweets_Label4=UK_News_V[UK_News_V['True Topic']=='Label4']
Tweets_Vec_Glove_lab4= np.concatenate(Tweets_Label4['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab4= np.concatenate(Tweets_Label4[gensim_embedXX].values)
# #Select some tweets ->lab 2
Tweets_Label2=UK_News_V[UK_News_V['True Topic']=='Label2']
Tweets_Vec_Glove_lab2= np.concatenate(Tweets_Label2['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab2= np.concatenate(Tweets_Label2[gensim_embedXX].values)
# #Select some tweets ->lab 1
Tweets_Label1=UK_News_V[UK_News_V['True Topic']=='Label1']
Tweets_Vec_Glove_lab1= np.concatenate(Tweets_Label1['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab1= np.concatenate(Tweets_Label1[gensim_embedXX].values)
# #Select some tweets ->lab 7
Tweets_Label7=UK_News_V[UK_News_V['True Topic']=='Label7']
Tweets_Vec_Glove_lab7= np.concatenate(Tweets_Label7['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab7= np.concatenate(Tweets_Label7[gensim_embedXX].values)
# #Select some tweets ->lab 3
Tweets_Label3=UK_News_V[UK_News_V['True Topic']=='Label3']
Tweets_Vec_Glove_lab3= np.concatenate(Tweets_Label3['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab3= np.concatenate(Tweets_Label3[gensim_embedXX].values)
# #Select some tweets ->lab 10
Tweets_Label10=UK_News_V[UK_News_V['True Topic']=='Label10']
Tweets_Vec_Glove_lab10= np.concatenate(Tweets_Label10['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab10= np.concatenate(Tweets_Label10[gensim_embedXX].values)
# #Select some tweets ->lab 12
Tweets_Label12=UK_News_V[UK_News_V['True Topic']=='Label12']
Tweets_Vec_Glove_lab12= np.concatenate(Tweets_Label12['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab12= np.concatenate(Tweets_Label12[gensim_embedXX].values)
# #Select some tweets ->lab 8_2
Tweets_Label8_2=UK_News_V[UK_News_V['True Topic']=='Label8_2']
Tweets_Vec_Glove_lab8_2= np.concatenate(Tweets_Label8_2['meanWordsGlove_tok_12'].values)
Tweets_Vec_Gensim_lab8_2= np.concatenate(Tweets_Label8_2[gensim_embedXX].values)

### Gensim Cosine Similarities for each topic - ' Similarity A'

In [835]:
gens_dim=50
gens_wind=8
Gensim_Tweets_Vec=[Tweets_Vec_Gensim_lab1,Tweets_Vec_Gensim_lab2,Tweets_Vec_Gensim_lab3,
                  Tweets_Vec_Gensim_lab4,Tweets_Vec_Gensim_lab4_2,Tweets_Vec_Gensim_lab6,
                  Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,
                  Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,
                  Tweets_Vec_Gensim_lab13]
means_simil_gesnim_ForEachLabel=[]
weights_gensim=[]
for i,j in zip(Gensim_Tweets_Vec,labels_list):
    gensim_CosMatrix=metrics.pairwise.cosine_similarity(i, Y=None, dense_output=True)
    gensim_CosMatrix2=gensim_CosMatrix[np.triu_indices(len(i), k = 1)]
    mean_simil_gens=round(np.mean(gensim_CosMatrix2),3)
    means_simil_gesnim_ForEachLabel.append(mean_simil_gens)
    weights_gensim.append(len(gensim_CosMatrix2))
    print('Gensim with dim and window =(',gens_dim,'and',gens_wind ,') the cosine mean similiraity for',j,'is',mean_simil_gens)
print(means_simil_gesnim_ForEachLabel)
Gensim_overall_simil=mean(means_simil_gesnim_ForEachLabel)
gensim_Weighted_overall_simil = np.average(means_simil_gesnim_ForEachLabel, weights=weights_gensim)
print('Gensim overall cosine similarity is =',round(Gensim_overall_simil,2))
print('Gensim Weighted overall cosine similarity is =',round(gensim_Weighted_overall_simil,2))

Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label1 is 0.815
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label2 is 0.823
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label3 is 0.725
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label4 is 0.879
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label4_2 is 0.794
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label6 is 0.787
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label7 is 0.878
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label8 is 0.814
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label8_2 is 0.76
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label9 is 0.693
Gensim with dim and window =( 50 and 8 ) the cosine mean similiraity for Label10 is 0.76
Gensim with dim an

### Glove Cosine Similarities for each topic - ' Similarity A'

In [836]:
vector_used1='Count Vectorizer' 
vector_used2='Tf-idf Vectorizer with idf=False'
Glove_Tweets_Vec=[Tweets_Vec_Glove_lab1,Tweets_Vec_Glove_lab2,Tweets_Vec_Glove_lab3,
                  Tweets_Vec_Glove_lab4,Tweets_Vec_Glove_lab4_2,Tweets_Vec_Glove_lab6,
                  Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,
                  Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,
                  Tweets_Vec_Glove_lab13]
means_simil_Glove_ForEachLabel=[]
weights_Glove=[]
print('Mittens Glove with co-occurence matrix using',vector_used1)
for i,j in zip(Glove_Tweets_Vec,labels_list):
    Glove_CosMatrix=metrics.pairwise.cosine_similarity(i, Y=None, dense_output=True)
    Glove_CosMatrix2=Glove_CosMatrix[np.triu_indices(len(i), k = 1)]
    mean_simil_Glove=round(np.mean(Glove_CosMatrix2),3)
    means_simil_Glove_ForEachLabel.append(mean_simil_Glove)
    weights_Glove.append(len(Glove_CosMatrix2))
    print('Glove with parm=',mittens_parm,' the cosine mean similiraity for',j,'is',mean_simil_Glove)
print(means_simil_Glove_ForEachLabel)
glove_overall_simil=mean(means_simil_Glove_ForEachLabel)
glove_Weighted_overall_simil = np.average(means_simil_Glove_ForEachLabel, weights=weights_Glove)
print('Glove overall cosine similarity is =',round(glove_overall_simil,2))
print('Glove Weighted overall cosine similarity is =',round(glove_Weighted_overall_simil,2))

Mittens Glove with co-occurence matrix using Count Vectorizer
Glove with parm= 0.1  the cosine mean similiraity for Label1 is 0.87
Glove with parm= 0.1  the cosine mean similiraity for Label2 is 0.844
Glove with parm= 0.1  the cosine mean similiraity for Label3 is 0.769
Glove with parm= 0.1  the cosine mean similiraity for Label4 is 0.85
Glove with parm= 0.1  the cosine mean similiraity for Label4_2 is 0.806
Glove with parm= 0.1  the cosine mean similiraity for Label6 is 0.8
Glove with parm= 0.1  the cosine mean similiraity for Label7 is 0.904
Glove with parm= 0.1  the cosine mean similiraity for Label8 is 0.806
Glove with parm= 0.1  the cosine mean similiraity for Label8_2 is 0.771
Glove with parm= 0.1  the cosine mean similiraity for Label9 is 0.78
Glove with parm= 0.1  the cosine mean similiraity for Label10 is 0.796
Glove with parm= 0.1  the cosine mean similiraity for Label12 is 0.788
Glove with parm= 0.1  the cosine mean similiraity for Label13 is 0.807
[0.87, 0.844, 0.769, 0.85,

In [837]:
#raname labels 
labels_list_new=['Label 1','Label 2','Label 3','Label 4','Label 5','Label 6','Label 7','Label 8','Label 9','Label 10','Label 11','Label 12','Label 13']
#Label -> Similarity
df_similarity_gens_glove= pd.DataFrame(columns =['Topic/Label','Similarity A (Gensim)','Similarity A (Glove)']) 
df_similarity_gens_glove['Topic/Label']=labels_list_new
df_similarity_gens_glove['Similarity A (Gensim)']=means_simil_gesnim_ForEachLabel
df_similarity_gens_glove['Similarity A (Glove)']=means_simil_Glove_ForEachLabel
df_similarity_gens_glove

,Topic/Label,Similarity A (Gensim),Similarity A (Glove)
0,Label 1,0.815,0.870
1,Label 2,0.823,0.844
2,Label 3,0.725,0.769
3,Label 4,0.879,0.850
4,Label 5,0.794,0.806
5,Label 6,0.787,0.800
6,Label 7,0.878,0.904
7,Label 8,0.814,0.806
8,Label 9,0.760,0.771
9,Label 10,0.693,0.780


### Notes about cosine mean similarity for the several mittens Glove model configuration and for the Gensim model that had been used for embedding tweets (on which clustering models have been fitted)
1) Glove overall Cosine mean Similarity with mittens  max_iter=2000,mittens=0.05 and dim=25 --> = 80%
<br>
2_a) Glove overall Cosine mean Similarity with mittens  max_iter=2000,mittens=0.1  and dim=25 --> = 82% 
<br>
2_b) Glove overall Cosine mean Similarity with countVectorizer and mittens  max_iter=2000,mittens=0.1  and dim=25 --> =81 % 
<br>
3) Glove overall Cosine mean Similarity with mittens  max_iter=2000,mittens=0.5  and dim=25 --> = 83%
<br>
4) Glove overall Cosine mean Similarity with mittens  max_iter=2000,mittens=1    and dim=25 --> = 83%
<br>
5) Glove overall Cosine mean Similarity with mittens  max_iter=2000,mittens=0.1  and dim=50 --> = 75%
<br>
6) Gensim  overall Cosine mean Similarity with dim=50, window=8                             --> = 79%

# Topic 'XX' vs all other topics -'Similarity B'
##### A low cosine mean similarity is a good performance since cosine similarity between tweets belonging to different clusters is expected to be low 

## Glove case

In [838]:
# lab 1 vs 2-13 
glo_lab1_vsAll=[]
for i in [Tweets_Vec_Glove_lab2,Tweets_Vec_Glove_lab3,Tweets_Vec_Glove_lab4,Tweets_Vec_Glove_lab4_2,Tweets_Vec_Glove_lab6,Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab1_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab1, Y=i, dense_output=True).mean())
# lab 2 vs 3-13
glo_lab2_vsAll=[]
for i in [Tweets_Vec_Glove_lab3,Tweets_Vec_Glove_lab4,Tweets_Vec_Glove_lab4_2,Tweets_Vec_Glove_lab6,Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab2_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab2, Y=i, dense_output=True).mean())

# lab 3 vs 4-13
glo_lab3_vsAll=[]
for i in [Tweets_Vec_Glove_lab4,Tweets_Vec_Glove_lab4_2,Tweets_Vec_Glove_lab6,Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab3_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab3, Y=i, dense_output=True).mean())
# lab 4 vs 4_2-13
glo_lab4_vsAll=[]
for i in [Tweets_Vec_Glove_lab4_2,Tweets_Vec_Glove_lab6,Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab4_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab4, Y=i, dense_output=True).mean())

# lab 4_2 vs 6-13
glo_lab4_2_vsAll=[]
for i in [Tweets_Vec_Glove_lab6,Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab4_2_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab4_2, Y=i, dense_output=True).mean())

# lab 6 vs 7-13
glo_lab6_vsAll=[]
for i in [Tweets_Vec_Glove_lab7,Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab6_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab6, Y=i, dense_output=True).mean())
# lab 7 vs 8-13
glo_lab7_vsAll=[]
for i in [Tweets_Vec_Glove_lab8,Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab7_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab7, Y=i, dense_output=True).mean())

# lab 8 vs 8_2-13
glo_lab8_vsAll=[]
for i in [Tweets_Vec_Glove_lab8_2,Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab8_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab8, Y=i, dense_output=True).mean())
# lab 8_2 vs 9-13
glo_lab8_2_vsAll=[]
for i in [Tweets_Vec_Glove_lab9,Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab8_2_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab8_2, Y=i, dense_output=True).mean())
# lab 9 vs 10-13
glo_lab9_vsAll=[]
for i in [Tweets_Vec_Glove_lab10,Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab9_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab9, Y=i, dense_output=True).mean())
# lab 10 vs 12-13
glo_lab10_vsAll=[]
for i in [Tweets_Vec_Glove_lab12,Tweets_Vec_Glove_lab13]:
    glo_lab10_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab10, Y=i, dense_output=True).mean())
# lab 12 vs 13
glo_lab12_vsAll=[]
for i in [Tweets_Vec_Glove_lab13]:
    glo_lab12_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Glove_lab12, Y=i, dense_output=True).mean())   

In [843]:
from statistics import mean 
list_print_support=['2-13','3-13','4-13','4_2-13','6-13','7-13','8-13','8_2-13','9-13','10-13','12-13','13']
glov_lab_vsAll=[glo_lab1_vsAll,glo_lab2_vsAll,glo_lab3_vsAll,glo_lab4_vsAll,
                glo_lab4_2_vsAll,glo_lab6_vsAll,glo_lab7_vsAll,glo_lab8_vsAll,
                glo_lab8_2_vsAll,glo_lab9_vsAll,glo_lab10_vsAll,glo_lab12_vsAll]
print('Glove with mittens parm equal to',mittens_parm,' Cosine mean similarity lab')
for i,j,h in zip(range(0,13),[1,2,3,4,'4_2',6,7,8,'8_2',9,10,12],list_print_support):
    print(j,'vs',h,'=',round(mean(glov_lab_vsAll[i]),2))
print('Glove overall:',mean([mean(glov_lab_vsAll[i]) for i in range(0,len(glov_lab_vsAll))]))   

Glove with mittens parm equal to 0.1  Cosine mean similarity lab
1 vs 2-13 = 0.63
2 vs 3-13 = 0.58
3 vs 4-13 = 0.63
4 vs 4_2-13 = 0.61
4_2 vs 6-13 = 0.61
6 vs 7-13 = 0.61
7 vs 8-13 = 0.7
8 vs 8_2-13 = 0.66
8_2 vs 9-13 = 0.66
9 vs 10-13 = 0.65
10 vs 12-13 = 0.72
12 vs 13 = 0.7
Glove overall: 0.6477335


In [830]:
glov_lab_vsAll_sum=glo_lab1_vsAll+glo_lab2_vsAll+glo_lab3_vsAll+glo_lab4_vsAll+glo_lab4_2_vsAll+glo_lab6_vsAll+glo_lab7_vsAll+glo_lab8_vsAll+glo_lab8_2_vsAll+glo_lab9_vsAll+glo_lab10_vsAll+glo_lab12_vsAll
print('Glove overall:',round(mean(glov_lab_vsAll_sum),3),'NB:by doing the mean of all cos_simil directly and not mean of means of lab xx vs all')

Glove overall: 0.631 NB:by doing the mean of all cos_simil directly and not mean of means of lab xx vs all


In [783]:
# Glove with mittens  max_iter=2000,mittens=0.5 and dim=25 -> Lab1 vs all others -> 68%
#                     -> Lab2 vs all others -> 63% -> Lab3 vs all others -> 69% -> Lab4 vs all others -> 65%
#                     -> Lab4_2 vs all others -> 67% -> Lab6 vs all others -> 64% -> Lab7 vs all others -> 72%
#                     -> Lab8 vs all others -> 71% -> Lab8_2 vs all others -> 71% -> Lab9 vs all others -> 69%
#                     -> Lab10 vs all others -> 76%  -> Lab12 vs all others -> 76% 
#                    OVERALL -> 69% (67.5% by doing the mean of all cos simil directly and not mean of means of lab xx vs all
# Glove with mittens  max_iter=2000,mittens=0.1 and dim=25 -> Lab1 vs all others -> 64%
#                     -> Lab2 vs all others -> 59% -> Lab3 vs all others -> 63% -> Lab4 vs all others -> 59%
#                     -> Lab4_2 vs all others -> 60% -> Lab6 vs all others -> 59% -> Lab7 vs all others -> 69%
#                     -> Lab8 vs all others -> 66% -> Lab8_2 vs all others -> 66% -> Lab9 vs all others -> 64%
#                     -> Lab10 vs all others -> 71%  -> Lab12 vs all others -> 71% 
#                    OVERALL -> 64% (62.5% by doing the mean of all cos simil directly ...)
# Glove with mittens with CountVectorizer  max_iter=2000,mittens=0.1 and dim=25 ->  lab 1 vs all: 0.64 
#                     ->lab 2 vs all: 0.59 lab 3 vs all: 0.64 lab 4 vs all: 0.61 
#                     ->lab 4_2 vs all: 0.62 lab 6 vs all: 0.61 lab 7 vs all: 0.7 
#                     ->lab 8 vs all: 0.67 lab 8_2 vs all: 0.66 lab 9 vs all: 0.65 
#                     ->lab 10 vs all: 0.72 lab 12 vs all: 0.71
#                    OVERALL ->65% (63% by doing the mean of all cos simil directly and not mean of means of lab xx vs all)
# Glove with mittens  max_iter=2000,mittens=0.05 and dim=25 ->lab 1 vs all: 0.62 ;lab 2 vs all: 0.56 
#                     ->lab 3 vs all: 0.6; lab 4 vs all: 0.56; lab 4_2 vs all: 0.57; lab 6 vs all: 0.56;
#                     ->lab 7 vs all: 0.66; lab 8 vs all: 0.64; lab 8_2 vs all: 0.64; lab 9 vs all: 0.62;
#                     -> lab 10 vs all: 0.69; lab 12 vs all: 0.67
#                    OVERALL -> 61%( 59.6% by doing the mean of all ....)
# Glove with mittens  max_iter=2000,mittens=1 and dim=25 ->lab 1 vs all: 0.7 lab 2 vs all: 0.65 
#                      ->lab 3 vs all: 0.71 lab 4 vs all: 0.67 lab 4_2 vs all: 0.7 lab 6 vs all: 0.66 
#                      ->lab 7 vs all: 0.73 lab 8 vs all: 0.73 lab 8_2 vs all: 0.73 lab 9 vs all: 0.74 
#                      ->lab 10 vs all: 0.78 lab 12 vs all: 0.78
#                    OVERALL -> 71.4%( 69.6% by doing the mean of all cos simil directly and not mean of means of lab xx vs all)
# Glove with mittens  max_iter=2000, mittens=0.1 and dim=50 -> 
#                    OVERALL -> 57.74% (56.3% by doing the mean of all cos simil directly ... )

#### Inspecting cosine similarities of all pair of different topics for Mittens Glove Model
-> with co-occurence matrix using 'case 2', dim=25 and mittens param=0.1

In [756]:
glov_lab_vsAll=[glo_lab1_vsAll,glo_lab2_vsAll,glo_lab3_vsAll,glo_lab4_vsAll,
                glo_lab4_2_vsAll,glo_lab6_vsAll,glo_lab7_vsAll,glo_lab8_vsAll,
                glo_lab8_2_vsAll,glo_lab9_vsAll,glo_lab10_vsAll,glo_lab12_vsAll]
#using the following label annotation for a more readbel output
lab_vsAll_names0=['Lab 2','Lab 3','Lab 4', 'Lab 5','Lab 6','Lab 7','Lab 8','Lab 9','Lab 10','Lab 11','Lab 12','Lab 13']
df_lab_vsALL_glov= pd.DataFrame(data=glov_lab_vsAll,columns =lab_vsAll_names0)
df_lab_vsALL_glov=df_lab_vsALL_glov.round(2)
df_lab_vsALL_glov=df_lab_vsALL_glov.replace(np.nan,'', regex=True)

In [757]:
Glov_lab2_vsAll2=df_lab_vsALL_glov.iloc[1].shift(periods=1, axis=0,fill_value='')
Glov_lab3_vsAll2=df_lab_vsALL_glov.iloc[2].shift(periods=2, axis=0,fill_value='')
Glov_lab4_vsAll2=df_lab_vsALL_glov.iloc[3].shift(periods=3, axis=0,fill_value='')
Glov_lab4_2_vsAll2=df_lab_vsALL_glov.iloc[4].shift(periods=4, axis=0,fill_value='')
Glov_lab6_vsAll2=df_lab_vsALL_glov.iloc[5].shift(periods=5, axis=0,fill_value='')
Glov_lab7_vsAll2=df_lab_vsALL_glov.iloc[6].shift(periods=6, axis=0,fill_value='')
Glov_lab8_vsAll2=df_lab_vsALL_glov.iloc[7].shift(periods=7, axis=0,fill_value='')
Glov_lab8_2_vsAll2=df_lab_vsALL_glov.iloc[8].shift(periods=8, axis=0,fill_value='')
Glov_lab9_vsAll2=df_lab_vsALL_glov.iloc[9].shift(periods=9, axis=0,fill_value='')
Glov_lab10_vsAll2=df_lab_vsALL_glov.iloc[10].shift(periods=10, axis=0,fill_value='')
Glov_lab12_vsAll2=df_lab_vsALL_glov.iloc[11].shift(periods=11, axis=0,fill_value='')

Glov_lab_vsAll2=[Glov_lab2_vsAll2,Glov_lab3_vsAll2,Glov_lab4_vsAll2,Glov_lab4_2_vsAll2,Glov_lab6_vsAll2,Glov_lab7_vsAll2,Glov_lab8_vsAll2,
                Glov_lab8_2_vsAll2,Glov_lab9_vsAll2,Glov_lab10_vsAll2,Glov_lab12_vsAll2]

for i,j in zip(Glov_lab_vsAll2,[1,2,3,4,5,6,7,8,9,10,11]):
    df_lab_vsALL_glov.iloc[j]=i
lab_vsAll_names1=['Lab 1 vs','Lab 2 vs','Lab 3 vs','Lab 4 vs', 'Lab 5 vs','Lab 6 vs','Lab 7 vs',
                'Lab 8 vs','Lab 9 vs','Lab 10 vs','Lab 11 vs','Lab 12 vs']
df_lab_vsALL_glov.insert(loc=0, column='Lab X vs Lab Y', value=lab_vsAll_names1)
df_lab_vsALL_glov['Similarities mean']=[mean(glov_lab_vsAll[i]) for i in range(0,len(glov_lab_vsAll))]
df_lab_vsALL_glov=df_lab_vsALL_glov.round(2)

In [758]:
df_lab_vsALL_glov

,Lab X vs Lab Y,Lab 2,Lab 3,Lab 4,Lab 5,Lab 6,Lab 7,Lab 8,Lab 9,Lab 10,Lab 11,Lab 12,Lab 13,Similarities mean
0,Lab 1 vs,0.61,0.65,0.57,0.63,0.62,0.66,0.59,0.61,0.67,0.71,0.68,0.69,0.64
1,Lab 2 vs,,0.58,0.53,0.55,0.57,0.62,0.54,0.57,0.6,0.62,0.67,0.58,0.59
2,Lab 3 vs,,,0.61,0.66,0.62,0.61,0.63,0.65,0.64,0.65,0.7,0.63,0.64
3,Lab 4 vs,,,,0.69,0.63,0.58,0.59,0.63,0.56,0.61,0.62,0.6,0.61
4,Lab 5 vs,,,,,0.65,0.57,0.6,0.63,0.61,0.64,0.63,0.63,0.62
5,Lab 6 vs,,,,,,0.63,0.6,0.59,0.62,0.63,0.62,0.59,0.61
6,Lab 7 vs,,,,,,,0.67,0.66,0.7,0.73,0.71,0.73,0.70
7,Lab 8 vs,,,,,,,,0.72,0.56,0.69,0.68,0.67,0.67
8,Lab 9 vs,,,,,,,,,0.6,0.72,0.68,0.64,0.66
9,Lab 10 vs,,,,,,,,,,0.67,0.68,0.61,0.65


## Gensim case

In [844]:
# lab 1 vs 
Gen_lab1_vsAll=[]
for i in [Tweets_Vec_Gensim_lab2,Tweets_Vec_Gensim_lab3,Tweets_Vec_Gensim_lab4,Tweets_Vec_Gensim_lab4_2,Tweets_Vec_Gensim_lab6,Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab1_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab1, Y=i, dense_output=True).mean())
# lab 2 vs 
Gen_lab2_vsAll=[]
for i in [Tweets_Vec_Gensim_lab3,Tweets_Vec_Gensim_lab4,Tweets_Vec_Gensim_lab4_2,Tweets_Vec_Gensim_lab6,Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab2_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab2, Y=i, dense_output=True).mean())

# lab 3 vs 
Gen_lab3_vsAll=[]
for i in [Tweets_Vec_Gensim_lab4,Tweets_Vec_Gensim_lab4_2,Tweets_Vec_Gensim_lab6,Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab3_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab3, Y=i, dense_output=True).mean())
# lab 4 vs 
Gen_lab4_vsAll=[]
for i in [Tweets_Vec_Gensim_lab4_2,Tweets_Vec_Gensim_lab6,Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab4_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab4, Y=i, dense_output=True).mean())

# lab 4_2 vs 
Gen_lab4_2_vsAll=[]
for i in [Tweets_Vec_Gensim_lab6,Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab4_2_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab4_2, Y=i, dense_output=True).mean())

# lab 6 vs 
Gen_lab6_vsAll=[]
for i in [Tweets_Vec_Gensim_lab7,Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab6_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab6, Y=i, dense_output=True).mean())
# lab 7 vs 
Gen_lab7_vsAll=[]
for i in [Tweets_Vec_Gensim_lab8,Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab7_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab7, Y=i, dense_output=True).mean())

# lab 8 vs 
Gen_lab8_vsAll=[]
for i in [Tweets_Vec_Gensim_lab8_2,Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab8_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab8, Y=i, dense_output=True).mean())
# lab 8_2 vs 
Gen_lab8_2_vsAll=[]
for i in [Tweets_Vec_Gensim_lab9,Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab8_2_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab8_2, Y=i, dense_output=True).mean())
# lab 9 vs 
Gen_lab9_vsAll=[]
for i in [Tweets_Vec_Gensim_lab10,Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab9_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab9, Y=i, dense_output=True).mean())
# lab 10 vs 
Gen_lab10_vsAll=[]
for i in [Tweets_Vec_Gensim_lab12,Tweets_Vec_Gensim_lab13]:
    Gen_lab10_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab10, Y=i, dense_output=True).mean())
# lab 12 vs 
Gen_lab12_vsAll=[]
for i in [Tweets_Vec_Gensim_lab13]:
    Gen_lab12_vsAll.append(metrics.pairwise.cosine_similarity(Tweets_Vec_Gensim_lab12, Y=i, dense_output=True).mean())   

In [851]:
from statistics import mean 
gens_lab_vsAll=[Gen_lab1_vsAll,Gen_lab2_vsAll,Gen_lab3_vsAll,Gen_lab4_vsAll,Gen_lab4_2_vsAll,Gen_lab6_vsAll,Gen_lab7_vsAll,Gen_lab8_vsAll,
                Gen_lab8_2_vsAll,Gen_lab9_vsAll,Gen_lab10_vsAll,Gen_lab12_vsAll]

print('Gensim Cosine mean similarity lab')
for i,j,h in zip(range(0,13),[1,2,3,4,'4_2',6,7,8,'8_2',9,10,12],list_print_support):
    print(j,'vs',h,'=',round(mean(gens_lab_vsAll[i]),2))
print('gensim overall:',mean([mean(gens_lab_vsAll[i]) for i in range(0,len(gens_lab_vsAll))]))   

Gensim Cosine mean similarity lab
1 vs 2-13 = 0.2
2 vs 3-13 = 0.19
3 vs 4-13 = 0.24
4 vs 4_2-13 = 0.18
4_2 vs 6-13 = 0.1
6 vs 7-13 = 0.22
7 vs 8-13 = 0.38
8 vs 8_2-13 = 0.43
8_2 vs 9-13 = 0.36
9 vs 10-13 = 0.27
10 vs 12-13 = 0.47
12 vs 13 = 0.51
gensim overall: 0.29690757


In [846]:
Gen_lab_vsAll_sum=Gen_lab1_vsAll+Gen_lab2_vsAll+Gen_lab3_vsAll+Gen_lab4_vsAll+Gen_lab4_2_vsAll+Gen_lab6_vsAll+glo_lab7_vsAll+Gen_lab8_vsAll+Gen_lab8_2_vsAll+Gen_lab9_vsAll+Gen_lab10_vsAll+Gen_lab12_vsAll
print('gensim overall:',round(mean(Gen_lab_vsAll_sum),2),'NB: by doing the mean of all cos_simil directly')

gensim overall: 0.27 NB: by doing the mean of all cos_simil directly


In [ ]:
# Gensim with dim=50 and window=8  -> Lab1 vs all others -> 20%
#                     -> Lab2 vs all others -> 19% -> Lab3 vs all others -> 24% -> Lab4 vs all others -> 18%
#                     -> Lab4_2 vs all others -> 10% -> Lab6 vs all others -> 22% -> Lab7 vs all others -> 38%
#                     -> Lab8 vs all others -> 43% -> Lab8_2 vs all others -> 36% -> Lab9 vs all others -> 27%
#                     -> Lab10 vs all others -> 47%  -> Lab12 vs all others -> 51% 
#                    OVERALL -> 29% (27% by doing the mean of all cos simil directly and nor mean of means of lab xx ve all))
# Gensim with dim=25 and window=8  ->lab 1 vs all: 0.21 lab 2 vs all: 0.2 lab 3 vs all: 0.24 lab 4 vs all: 0.17 
#                      ->lab 4_2 vs all: 0.1 lab 6 vs all: 0.22 lab 7 vs all: 0.39 lab 8 vs all: 0.44 
#                      ->lab 8_2 vs all: 0.37 lab 9 vs all: 0.3 lab 10 vs all: 0.47 lab 12 vs all: 0.5
#                    OVERALL -> 30% (26% by doing the mean of all cos simil directly... )
# Gensim with dim=100 and window=8  ->lab 1 vs all: 0.21 lab 2 vs all: 0.2 lab 3 vs all: 0.24 lab 4 vs all: 0.19 
#                    ->lab 4_2 vs all: 0.1 lab 6 vs all: 0.22 lab 7 vs all: 0.4 lab 8 vs all: 0.44 
#                    ->lab 8_2 vs all: 0.37 lab 9 vs all: 0.28 lab 10 vs all: 0.48 lab 12 vs all: 0.52
#                    OVERALL -> 30% (27% by doing the mean of all cos simil directly... )
# Gensim with dim=50 and window=4  ->lab 1 vs all: 0.26 lab 2 vs all: 0.24 lab 3 vs all: 0.3 lab 4 vs all: 0.22 
#                  ->lab 4_2 vs all: 0.16 lab 6 vs all: 0.28 lab 7 vs all: 0.45 lab 8 vs all: 0.48 
#                  ->lab 8_2 vs all: 0.42 lab 9 vs all: 0.34 lab 10 vs all: 0.52 lab 12 vs all: 0.54
#                    OVERALL -> 35% (31% by doing the mean of all cos simil directly....)
#Gensim with dim=50 and window=12  ->
#                    OVERALL -> 29% (26% by doing the mean of all cos simil directly....)

#### Inspecting cosine similarities of all pair of different topics for Gensim Model
-> with dim=50 and window=8

In [760]:
gens_lab_vsAll=[Gen_lab1_vsAll,Gen_lab2_vsAll,Gen_lab3_vsAll,Gen_lab4_vsAll,Gen_lab4_2_vsAll,Gen_lab6_vsAll,Gen_lab7_vsAll,Gen_lab8_vsAll,
                Gen_lab8_2_vsAll,Gen_lab9_vsAll,Gen_lab10_vsAll,Gen_lab12_vsAll]
lab_vsAll_names0=['Lab 2','Lab 3','Lab 4', 'Lab 5','Lab 6','Lab 7','Lab 8','Lab 9','Lab 10','Lab 11','Lab 12','Lab 13']
df_lab_vsALL_Gen= pd.DataFrame(data=gens_lab_vsAll,columns =lab_vsAll_names0)
df_lab_vsALL_Gen=df_lab_vsALL_Gen.round(2)
df_lab_vsALL_Gen=df_lab_vsALL_Gen.replace(np.nan,'', regex=True)

In [761]:
Gen_lab2_vsAll2=df_lab_vsALL_Gen.iloc[1].shift(periods=1, axis=0,fill_value='')
Gen_lab3_vsAll2=df_lab_vsALL_Gen.iloc[2].shift(periods=2, axis=0,fill_value='')
Gen_lab4_vsAll2=df_lab_vsALL_Gen.iloc[3].shift(periods=3, axis=0,fill_value='')
Gen_lab4_2_vsAll2=df_lab_vsALL_Gen.iloc[4].shift(periods=4, axis=0,fill_value='')
Gen_lab6_vsAll2=df_lab_vsALL_Gen.iloc[5].shift(periods=5, axis=0,fill_value='')
Gen_lab7_vsAll2=df_lab_vsALL_Gen.iloc[6].shift(periods=6, axis=0,fill_value='')
Gen_lab8_vsAll2=df_lab_vsALL_Gen.iloc[7].shift(periods=7, axis=0,fill_value='')
Gen_lab8_2_vsAll2=df_lab_vsALL_Gen.iloc[8].shift(periods=8, axis=0,fill_value='')
Gen_lab9_vsAll2=df_lab_vsALL_Gen.iloc[9].shift(periods=9, axis=0,fill_value='')
Gen_lab10_vsAll2=df_lab_vsALL_Gen.iloc[10].shift(periods=10, axis=0,fill_value='')
Gen_lab12_vsAll2=df_lab_vsALL_Gen.iloc[11].shift(periods=11, axis=0,fill_value='')

gens_lab_vsAll2=[Gen_lab2_vsAll2,Gen_lab3_vsAll2,Gen_lab4_vsAll2,Gen_lab4_2_vsAll2,Gen_lab6_vsAll2,Gen_lab7_vsAll2,Gen_lab8_vsAll2,
                Gen_lab8_2_vsAll2,Gen_lab9_vsAll2,Gen_lab10_vsAll2,Gen_lab12_vsAll2]

for i,j in zip(gens_lab_vsAll2,[1,2,3,4,5,6,7,8,9,10,11]):
    df_lab_vsALL_Gen.iloc[j]=i
lab_vsAll_names1=['Lab 1 vs','Lab 2 vs','Lab 3 vs','Lab 4 vs', 'Lab 5 vs','Lab 6 vs','Lab 7 vs',
                'Lab 8 vs','Lab 9 vs','Lab 10 vs','Lab 11 vs','Lab 12 vs']
df_lab_vsALL_Gen.insert(loc=0, column='Lab X vs Lab Y', value=lab_vsAll_names1)
df_lab_vsALL_Gen['Similarities mean']=[mean(gens_lab_vsAll[i]) for i in range(0,len(gens_lab_vsAll))]
df_lab_vsALL_Gen=df_lab_vsALL_Gen.round(2)
df_lab_vsALL_Gen

,Lab X vs Lab Y,Lab 2,Lab 3,Lab 4,Lab 5,Lab 6,Lab 7,Lab 8,Lab 9,Lab 10,Lab 11,Lab 12,Lab 13,Similarities mean
0,Lab 1 vs,0.2,0.2,-0.01,0.1,0.18,0.18,0.11,0.1,0.28,0.35,0.23,0.49,0.20
1,Lab 2 vs,,0.18,0.06,0.01,0.12,0.24,0.25,0.22,0.25,0.22,0.37,0.21,0.19
2,Lab 3 vs,,,0.2,0.27,0.18,0.08,0.29,0.23,0.25,0.15,0.45,0.28,0.24
3,Lab 4 vs,,,,0.41,0.17,0.2,0.16,0.27,0.2,0.12,0.07,0.03,0.18
4,Lab 5 vs,,,,,0.29,-0.04,0.15,0.19,0.07,0.01,0.01,0.12,0.10
5,Lab 6 vs,,,,,,0.24,0.22,0.19,0.28,0.22,0.17,0.23,0.22
6,Lab 7 vs,,,,,,,0.25,0.3,0.4,0.47,0.42,0.44,0.38
7,Lab 8 vs,,,,,,,,0.74,-0,0.52,0.45,0.46,0.43
8,Lab 9 vs,,,,,,,,,0.1,0.58,0.36,0.39,0.36
9,Lab 10 vs,,,,,,,,,,0.31,0.33,0.17,0.27


# Cosine similarities for all Models 

In [847]:
#Model -> dim -> parmeter -> Similarity
df_similarity= pd.DataFrame(columns =['Model', 'dim','parameter','Similarity A','Similarity B']) 
df_similarity['Model']=['Mittens Glove','Mittens Glove (a)','Mittens Glove (b)','Mittens Glove','Mittens Glove','Mittens Glove','Trained Gensim','Trained Gensim','Trained Gensim','Trained Gensim','Trained Gensim']
df_similarity['dim']=[25,25,25,25,25,50,50,25,100,50,50]
df_similarity['parameter']=[0.05,0.1,0.1,0.5,1,0.1,8,8,8,4,12]
df_similarity['Similarity A']=[0.80,0.82,0.83,0.83,0.83,0.75,0.79,simil_means2_GensimModels_all[2],
                               simil_means2_GensimModels_all[3],simil_means2_GensimModels_all[0],
                               simil_means2_GensimModels_all[1]]
df_similarity['Similarity B']=[0.60,0.63,0.64,0.68,0.70,0.56,0.27,0.26,0.27,0.31,0.26]
df_similarity

,Model,dim,parameter,Similarity A,Similarity B
0,Mittens Glove,25,0.05,0.80,0.60
1,Mittens Glove (a),25,0.10,0.82,0.63
2,Mittens Glove (b),25,0.10,0.83,0.64
3,Mittens Glove,25,0.50,0.83,0.68
4,Mittens Glove,25,1.00,0.83,0.70
5,Mittens Glove,50,0.10,0.75,0.56
6,Trained Gensim,50,8.00,0.79,0.27
7,Trained Gensim,25,8.00,0.79,0.26
8,Trained Gensim,100,8.00,0.79,0.27
9,Trained Gensim,50,4.00,0.79,0.31
